In [1]:
import pandas as pd
import numpy as np
import random

csv = pd.read_csv('res/books.csv', sep=";")
books = csv[["Nepali Title", "English Title"]]


FileNotFoundError: [Errno 2] No such file or directory: 'res/books.csv'

In [ ]:
NUM_USERS = 100
NUM_BOOKS, _ = books.shape

def get_ratings_by_user(expected_num, num_books):
    ratings = []
    for _ in range(num_books):
        if random.randint(1, 100) <= expected_num * 100:
            rating = random.randint(1, 11)
            ratings.append(rating)
        else:
            ratings.append(0)
    return ratings

rating_array = [get_ratings_by_user(random.uniform(0.2, 0.4), NUM_BOOKS) for _ in range(NUM_USERS)]
rating_matrix = np.array(rating_array, dtype='int')
rating_df = pd.DataFrame(rating_matrix)
print(rating_df)

    0    1    2    3    4    5    6    7    8    9    ...  396  397  398  399  \
0    11    0    0    7    0    3    0    9    6    0  ...    0    0    9    3   
1     0    0    0    0    3    3    0    0    0    0  ...    0    0    4    0   
2     0    0    5    0    0    4    0    0    0    0  ...    2    0    0    0   
3     0    0    0    1    0    8    5    0    0    7  ...    0    0    0    0   
4     0    0    0    1    9    0   10    0    8    0  ...    0    0    0    0   
..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
95    0    0    0    0    0    8    0    0    8    0  ...    5    0    0    0   
96    0    0    0    0    0    0    7    5    4    0  ...    0    3    0    0   
97    0    0    0    0    0    3    8    0    0    0  ...    0    6   10    5   
98    0    0    0    3    0    2    6    0    0    0  ...    9    2    0    6   
99    6    0    4    9    8    7   10    1    0    0  ...    0    7    0    0   

    400  401  402  403  404

In [ ]:

def get_estimated_rating_for(USER_ID, BOOK_ID):
    users_who_have_rated_book = rating_df[rating_df[BOOK_ID] != 0].index
    users_who_have_rated_book_ratings = rating_df.iloc[users_who_have_rated_book].copy(deep=True)
    user_rating = rating_df.iloc[USER_ID]
    users_who_have_rated_book_ratings.loc[USER_ID] = user_rating
    users_who_have_rated_book_ratings = users_who_have_rated_book_ratings.transpose()
    original_axes = users_who_have_rated_book_ratings.axes[1].to_list()


    def normalize_matrix(row, user_rating):
        return (row * user_rating / user_rating).fillna(0)
    r = users_who_have_rated_book_ratings.apply(lambda x: normalize_matrix(x, user_rating)).transpose()

    from sklearn.metrics.pairwise import cosine_similarity
    similarity_scores = cosine_similarity(r)
    similarity_scores = pd.DataFrame(similarity_scores)

    users_who_have_rated_book_ratings = rating_df.iloc[users_who_have_rated_book]

    similarity_scores = similarity_scores.set_axis(original_axes, axis="index")
    similarity_scores = similarity_scores.set_axis(original_axes, axis="columns")


    average_data = []
    for u in original_axes:
        if u != USER_ID:
            weight = similarity_scores[USER_ID][u]
            rating = users_who_have_rated_book_ratings[BOOK_ID][u]
            average_data.append((rating, weight))

    ratings, weights = zip(*average_data)
    estimated_rating = sum(map(lambda x: x[0] * x[1], average_data)) / sum(weights)
    return estimated_rating

ratings = []
for i in range(400):
    ratings.append((get_estimated_rating_for(50, i), i))

print(sorted(ratings))

[(4.112177529929353, 132), (4.323844628505165, 383), (4.506809487965616, 244), (4.541894246305333, 238), (4.5492080181290655, 183), (4.577045291251009, 43), (4.621096481915048, 375), (4.627297828664445, 304), (4.691131272552665, 286), (4.735905340362898, 28), (4.7898908790104775, 184), (4.807218995875377, 284), (4.8201093561002, 9), (4.844637920990847, 168), (4.893224905357968, 118), (4.912645899081385, 252), (4.929118066042637, 258), (4.93355643708733, 268), (4.976002480632411, 306), (4.9802188290817515, 62), (4.992666878242465, 123), (5.007590160777739, 171), (5.016935299856463, 40), (5.054291558853335, 301), (5.0574561437728365, 196), (5.068488044596477, 271), (5.1167241709540425, 345), (5.127347308266401, 320), (5.145711247054613, 195), (5.153874825966842, 3), (5.154406680553764, 80), (5.1549695177635755, 68), (5.158701752121018, 257), (5.188806157207008, 198), (5.19322375961219, 10), (5.2074544188864875, 94), (5.208688367096399, 310), (5.213775581359646, 103), (5.220360737857472, 